In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
import torch
model_name = 'meta-llama/Llama-3.2-3B'
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = AutoModelForCausalLM.from_pretrained(model_name,
                             quantization_config = bnb_config,
                             device_map = DEVICE,
                                             token="")#Put your own HF token here
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'




In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
import torch
tokenizer = AutoTokenizer.from_pretrained("Harsh0910/my_merged_model")
model = AutoModelForCausalLM.from_pretrained("Harsh0910/my_merged_model")
input_texts = [
    "Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday. Detectives said three firearms, ammunition, and a five-figure sum of money were recovered. A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday.",
    "Jordan Hill, Brittany Covington, and Tesfaye Cooper, all 18, and Tanishia Covington, 24, appeared in a Chicago court on Friday. The four have been charged with hate crimes, aggravated kidnapping, and battery, among other things. An online fundraiser for their victim has collected $51,000 (£42,500) so far. Denying the four suspects bail, Judge Maria Kuriakos Ciesil asked, 'Where was your sense of decency?' Prosecutors told the court the beating started in a van and continued at a house, where the suspects allegedly forced the 18-year-old white victim, who suffers from schizophrenia and attention deficit disorder, to drink toilet water and kiss the floor. Police allege the van was earlier stolen by Mr. Hill, who is also accused of demanding $300 from the victim's mother while they held him captive. The court was also told the suspects stuffed a sock into his mouth, taped his mouth shut, and bound his hands with a belt. In a video made for Facebook Live, watched millions of times, the assailants can be heard making derogatory statements against white people and Donald Trump. The victim had been dropped off at a McDonald's to meet Mr. Hill, one of his friends, on 31 December. He was found by a police officer on Tuesday, 3 January, a day after he was reported missing by his parents. Prosecutors say the suspects each face two hate crimes counts: one because of the victim's race and the other because of his disabilities.",
    "The 48-year-old former Arsenal goalkeeper played for the Royals for four years. He was appointed youth academy director in 2000 and has been director of football since 2003. A West Brom statement said: 'He played a key role in the Championship club twice winning promotion to the Premier League in 2006 and 2012.'"
]


reference_summaries = [
    "A man has appeared in court after firearms, ammunition, and cash were seized by police in Edinburgh.",
    "Four people accused of kidnapping and torturing a mentally disabled man in a 'racially motivated' attack streamed on Facebook have been denied bail.",
    "West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading."
]
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
all_scores = []
for input_text, reference_summary in zip(input_texts, reference_summaries):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    output_ids = model.generate(inputs['input_ids'], max_length=400, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    scores = scorer.score(reference_summary, generated_summary)
    all_scores.append(scores)
    
    print(f"Input Text: {input_text}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {generated_summary}")
    print(f"ROUGE Scores: {scores}")
    print()

average_scores = {
    metric: sum(score[metric].fmeasure for score in all_scores) / len(all_scores)
    for metric in ['rouge1', 'rouge2', 'rougeL']
}

print("Average ROUGE Scores:")
for metric, score in average_scores.items():
    print(f"{metric.upper()}: {score:.4f}")


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


In [ ]:
def generate_summary(model, tokenizer, input_text, max_length=100):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

    outputs = model.generate(inputs['input_ids'], max_length=max_length, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


In [ ]:
def evaluate_rouge(model, tokenizer, input_texts, reference_summaries):
    results = []
    for input_text, reference_summary in zip(input_texts, reference_summaries):
        generated_summary = generate_summary(model, tokenizer, input_text)

        scores = scorer.score(reference_summary, generated_summary)
        results.append(scores)
    return results


In [ ]:
input_texts = [
    "Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday. Detectives said three firearms, ammunition, and a five-figure sum of money were recovered. A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday.",
    "Jordan Hill, Brittany Covington, and Tesfaye Cooper, all 18, and Tanishia Covington, 24, appeared in a Chicago court on Friday. The four have been charged with hate crimes, aggravated kidnapping, and battery, among other things. An online fundraiser for their victim has collected $51,000 (£42,500) so far. Denying the four suspects bail, Judge Maria Kuriakos Ciesil asked, 'Where was your sense of decency?' Prosecutors told the court the beating started in a van and continued at a house, where the suspects allegedly forced the 18-year-old white victim, who suffers from schizophrenia and attention deficit disorder, to drink toilet water and kiss the floor. Police allege the van was earlier stolen by Mr. Hill, who is also accused of demanding $300 from the victim's mother while they held him captive. The court was also told the suspects stuffed a sock into his mouth, taped his mouth shut, and bound his hands with a belt. In a video made for Facebook Live, watched millions of times, the assailants can be heard making derogatory statements against white people and Donald Trump. The victim had been dropped off at a McDonald's to meet Mr. Hill, one of his friends, on 31 December. He was found by a police officer on Tuesday, 3 January, a day after he was reported missing by his parents. Prosecutors say the suspects each face two hate crimes counts: one because of the victim's race and the other because of his disabilities.",
    "The 48-year-old former Arsenal goalkeeper played for the Royals for four years. He was appointed youth academy director in 2000 and has been director of football since 2003. A West Brom statement said: 'He played a key role in the Championship club twice winning promotion to the Premier League in 2006 and 2012.'"
]


reference_summaries = [
    "A man has appeared in court after firearms, ammunition, and cash were seized by police in Edinburgh.",
    "Four people accused of kidnapping and torturing a mentally disabled man in a 'racially motivated' attack streamed on Facebook have been denied bail.",
    "West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading."
]



In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

all_scores = []
for input_text, reference_summary in zip(input_texts, reference_summaries):
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    output_ids = model.generate(inputs, max_length=400, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    scores = scorer.score(reference_summary, generated_summary)
    all_scores.append(scores)
    
    print(f"Input Text: {input_text}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {generated_summary}")
    print(f"ROUGE Scores: {scores}")
    print()

average_scores = {
    metric: sum(score[metric].fmeasure for score in all_scores) / len(all_scores)
    for metric in ['rouge1', 'rouge2', 'rougeL']
}

print("Average ROUGE Scores:")
for metric, score in average_scores.items():
    print(f"{metric.upper()}: {score:.4f}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from rouge_score import rouge_scorer
import torch

model_name = 'meta-llama/Llama-3.2-3B'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model1 = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=DEVICE,
    token=""#Put your own HF token here
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

input_texts = [
    "Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday. Detectives said three firearms, ammunition, and a five-figure sum of money were recovered. A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday.",
    "Jordan Hill, Brittany Covington, and Tesfaye Cooper, all 18, and Tanishia Covington, 24, appeared in a Chicago court on Friday. The four have been charged with hate crimes, aggravated kidnapping, and battery, among other things. An online fundraiser for their victim has collected $51,000 (£42,500) so far. Denying the four suspects bail, Judge Maria Kuriakos Ciesil asked, 'Where was your sense of decency?' Prosecutors told the court the beating started in a van and continued at a house, where the suspects allegedly forced the 18-year-old white victim, who suffers from schizophrenia and attention deficit disorder, to drink toilet water and kiss the floor. Police allege the van was earlier stolen by Mr. Hill, who is also accused of demanding $300 from the victim's mother while they held him captive. The court was also told the suspects stuffed a sock into his mouth, taped his mouth shut, and bound his hands with a belt. In a video made for Facebook Live, watched millions of times, the assailants can be heard making derogatory statements against white people and Donald Trump. The victim had been dropped off at a McDonald's to meet Mr. Hill, one of his friends, on 31 December. He was found by a police officer on Tuesday, 3 January, a day after he was reported missing by his parents. Prosecutors say the suspects each face two hate crimes counts: one because of the victim's race and the other because of his disabilities.",
    "The 48-year-old former Arsenal goalkeeper played for the Royals for four years. He was appointed youth academy director in 2000 and has been director of football since 2003. A West Brom statement said: 'He played a key role in the Championship club twice winning promotion to the Premier League in 2006 and 2012.'"
]


reference_summaries = [
    "A man has appeared in court after firearms, ammunition, and cash were seized by police in Edinburgh.",
    "Four people accused of kidnapping and torturing a mentally disabled man in a 'racially motivated' attack streamed on Facebook have been denied bail.",
    "West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading."
]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

all_scores = []
for input_text, reference_summary in zip(input_texts, reference_summaries):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()} 
    output_ids = model1.generate(inputs['input_ids'], max_length=400, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    scores = scorer.score(reference_summary, generated_summary)
    all_scores.append(scores)
    
    print(f"Input Text: {input_text}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {generated_summary}")
    print(f"ROUGE Scores: {scores}")
    print()

average_scores = {
    metric: sum(score[metric].fmeasure for score in all_scores) / len(all_scores)
    for metric in ['rouge1', 'rouge2', 'rougeL']
}

print("Average ROUGE Scores:")
for metric, score in average_scores.items():
    print(f"{metric.upper()}: {score:.4f}")


In [ ]:
from huggingface_hub import login
login()